# **Capstone Project Notebook**

#### This is the final capstone project of **Enrico Pizzutilo**
It will be mainly used for the capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Second Assignement


### Part 1

In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 1.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."
html_data = requests.get(url).text

In [7]:
soup = BeautifulSoup(html_data, "html5lib")
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [56]:
#Used to check the tables present in soup --> leave # to compute them
#soup.find_all("tbody")
a=soup.find("tbody").find("tr")
print(a)
#for element in a.next_siblings:
#    print("-----------")
#    print(element)
#soup.find_all("td")

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


<generator object next_siblings at 0x7f4a252cb360>

#### Create the Dataframe
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [142]:
neighborhood_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
print(neighborhood_df)

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    if col != []:
        postcode = col[0].text
        borough = col[1].text
        neighborhood = col[2].text
        #print("{}, {}, {}".format(postcode, borough, neighborhood))
        neighborhood_df = neighborhood_df.append({'PostalCode':postcode, 'Borough':borough, 'Neighborhood':neighborhood}, ignore_index=True)

neighborhood_df

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
...,...,...,...
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n
285,M8Z,Etobicoke,South of Bloor\n


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [150]:
#drop the row with borough not assigned
neighborhood_df=neighborhood_df.drop(neighborhood_df[neighborhood_df['Borough']=='Not assigned'].index)
neighborhood_df
#neighborhood_df['Borough']=='Not assigned'

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West\n
282,M8Z,Etobicoke,Mimico NW\n
283,M8Z,Etobicoke,The Queensway West\n
284,M8Z,Etobicoke,Royal York South West\n


- If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [176]:
#check if there is any non assigned neighborhood
a = neighborhood_df[neighborhood_df['Neighborhood']=='Not assigned']
a

,PostalCode,Borough,Neighborhood


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [144]:
#testing the codes for the following loops
unique_postalcode_df = neighborhood_df[neighborhood_df['PostalCode']=='M8Z']
print(unique_postalcode_df)
neigh_list_same_pc = unique_postalcode_df['Neighborhood'].tolist()
neighborhoods = ''.join(neigh_list_same_pc).replace("\n", " , ")[0:-3]
borough = unique_postalcode_df['Borough'][unique_postalcode_df.index[0]]
print(borough, neighborhoods)

    PostalCode    Borough                Neighborhood
281        M8Z  Etobicoke  Kingsway Park South West\n
282        M8Z  Etobicoke                 Mimico NW\n
283        M8Z  Etobicoke        The Queensway West\n
284        M8Z  Etobicoke     Royal York South West\n
285        M8Z  Etobicoke            South of Bloor\n
Etobicoke Kingsway Park South West , Mimico NW , The Queensway West , Royal York South West , South of Bloor


In [239]:
#starting from the previous dataframe use a loop to have all the neighborhood with the same postal code in the same row

unique_postalcode_total_table = []
print(unique_postalcode_total_table)

for unique_postalcode in pd.unique(neighborhood_df["PostalCode"]):
    cell = {}
    unique_postalcode_df = neighborhood_df[neighborhood_df['PostalCode']==unique_postalcode]
    
    cell['PostalCode'] = unique_postalcode
    cell['Borough'] = unique_postalcode_df['Borough'][unique_postalcode_df.index[0]]
    cell['Neighborhood'] = ''.join(unique_postalcode_df['Neighborhood'].tolist()).replace("\n", " , ")[0:-3]
    
    #print(cell)
    unique_postalcode_total_table.append(cell)
    #print(unique_postalcode_total_table)
    
unique_postalcode_total_df = pd.DataFrame(unique_postalcode_total_table)
unique_postalcode_total_df

[]


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay , King's Mill Park , Kingsway Park ..."


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [178]:
unique_postalcode_total_df.shape

(103, 3)

### Part 2

In [196]:
!pip install geocoder # install geocoder

     |████████████████████████████████| 102kB 16.6MB/s ta 0:00:01


<details>
<summary><b>Code from Coursera</b></summary>
    
import geocoder # import geocoder

#initialize your variable to None
lat_lng_coords = None

#loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
    
</details>

In [312]:
latitude=[]
longitude=[]
for code in unique_postalcode_total_df['PostalCode']:
  
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

M3A [43.75245000000007, -79.32990999999998]
M4A [43.73057000000006, -79.31305999999995]
M5A [43.65512000000007, -79.36263999999994]
M6A [43.72327000000007, -79.45041999999995]
M7A [43.66253000000006, -79.39187999999996]
M9A [43.662630000000036, -79.52830999999998]
M1B [43.811390000000074, -79.19661999999994]
M3B [43.74923000000007, -79.36185999999998]
M4B [43.70718000000005, -79.31191999999999]
M5B [43.65739000000008, -79.37803999999994]
M6B [43.70687000000004, -79.44811999999996]
M9B [43.65034000000003, -79.55361999999997]
M1C [43.78574000000003, -79.15874999999994]
M3C [43.72168000000005, -79.34351999999996]
M4C [43.68970000000007, -79.30681999999996]
M5C [43.65215000000006, -79.37586999999996]
M6C [43.69211000000007, -79.43035999999995]
M9C [43.64857000000006, -79.57824999999997]
M1E [43.765750000000025, -79.17469999999997]
M4E [43.67709000000008, -79.29546999999997]
M5E [43.64536000000004, -79.37305999999995]
M6E [43.68784000000005, -79.45045999999996]
M1G [43.76812000000007, -79.2

In [303]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


<details>
<summary><b>Test with geolocator
    Check out here for few test</b></summary>

CLIENT_ID = 'G5FDVX33SSBZWOBZASSRZ4QNZNJGJO3MCC3FHX3XF2UKVVHG' # your Foursquare ID
CLIENT_SECRET = 'IGCQL0BIKF0RECQ3K2KMKZ2PXPONB0U3KLRGKWKGHYPOI2RX' # your Foursquare Secret
ACCESS_TOKEN = 'QA01YLXKJIJRUPCEIVTA5YIIFST3I5VVL0DERZZGXOYE1FLO' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
    
    #check the name of the neighborhood
postal_code = 'M5G'
postal_code_df=unique_postalcode_total_df.loc[unique_postalcode_total_df[unique_postalcode_total_df['PostalCode'] == postal_code].index]
postal_code_borough = postal_code_df['Borough'][postal_code_df.index[0]]
#for neighborhood in case of multiple was chosen only the first one
postal_code_neighborhood = postal_code_df['Neighborhood'][postal_code_df.index[0]].split(", ")[0] 

print (postal_code_neighborhood, postal_code_borough)
    
    #check with geolocator
address = '{}, {} , Toronto, Canada '.format(postal_code_neighborhood, postal_code_borough)
print(address)
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
type(latitude)


    #loop thorugh
for postal_code in unique_postalcode_total_df["PostalCode"]:
        postal_code_df=unique_postalcode_total_df.loc[unique_postalcode_total_df[unique_postalcode_total_df['PostalCode'] == postal_code].index]
        postal_code_borough = postal_code_df['Borough'][postal_code_df.index[0]]
        #for neighborhood in case of multiple was chosen only the first one
        postal_code_neighborhood = postal_code_df['Neighborhood'][postal_code_df.index[0]].split(", ")[0] 
        address = '{}, {} , Toronto, Canada '.format(postal_code_neighborhood, postal_code_borough)
        geolocator = Nominatim(user_agent="foursquare_agent")
        if geolocator.geocode(address) != []:
            latitude = geolocator.geocode(address).latitude
            longitude = geolocator.geocode(address).longitude
        print ('The Address for postal code {} is: \n       {} \n         lat: {}, lon: {}'.format(postal_code, address, latitude, longitude))

</details>

In [319]:
#check that they have the same length
print(len(latitude))
unique_postalcode_total_df.shape

103


(103, 3)

In [325]:
unique_postalcode_total_df['Latitude'] = latitude
unique_postalcode_total_df['Longitude'] = longitude
type(latitude)
unique_postalcode_total_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264
3,M6A,North York,"Lawrence Heights , Lawrence Manor",43.72327,-79.45042
4,M7A,Downtown Toronto,Queen's Park,43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.38544
101,M8Y,Etobicoke,"Humber Bay , King's Mill Park , Kingsway Park ...",43.63278,-79.48945
